<a href="https://colab.research.google.com/github/2019mohamed/Free-Short-Answers-Assessment/blob/main/student%E2%80%99s_response_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
tree = ET.parse('/content/drive/MyDrive/grade_data.xml')

In [ ]:
root = tree.getroot()

Explore the dataset

In [ ]:
[elem.tag for elem in root.iter()]


['root',
 'Instance',
 'MetaInfo',
 'ProblemDescription',
 'Question',
 'Answer',
 'Annotation',
 'AdditionalAnnotation',
 'Comments',
 'ReferenceAnswers',
 'Instance',
 'MetaInfo',
 'ProblemDescription',
 'Question',
 'Answer',
 'Annotation',
 'AdditionalAnnotation',
 'Comments',
 'ReferenceAnswers',
 'Instance',
 'MetaInfo',
 'ProblemDescription',
 'Question',
 'Answer',
 'Annotation',
 'AdditionalAnnotation',
 'Comments',
 'ReferenceAnswers',
 'Instance',
 'MetaInfo',
 'ProblemDescription',
 'Question',
 'Answer',
 'Annotation',
 'AdditionalAnnotation',
 'Comments',
 'ReferenceAnswers',
 'Instance',
 'MetaInfo',
 'ProblemDescription',
 'Question',
 'Answer',
 'Annotation',
 'AdditionalAnnotation',
 'Comments',
 'ReferenceAnswers',
 'Instance',
 'MetaInfo',
 'ProblemDescription',
 'Question',
 'Answer',
 'Annotation',
 'AdditionalAnnotation',
 'Comments',
 'ReferenceAnswers',
 'Instance',
 'MetaInfo',
 'ProblemDescription',
 'Question',
 'Answer',
 'Annotation',
 'AdditionalAnnotatio

In [ ]:
for q in root.iter('Answer'):
    print(q.text)
    print('--------')

the windshield will apply a force to the mosquito equal the the force applied by the mosquito to the windshield
--------
The normal force coming from the ice and the gravitational force. The velocity of the puck is constant because there are no counteracting forces acting on the puck. As stated by the first law of Newton, an object will maintain its state of motion unless acted upon by a force.
--------
if there is a zero net force on the object, the velocity will be constant
--------
An equal force always balancing it out regardless of the different masses and weights.
--------
The speed of the puck will equal to the net force times the mass of the puck
--------
The meteor will continue in a straight line with a constant velocity until another force acts upon it. This is due to newtons first law of motion, objects at constant velocity will stay so unless a net force acts upon it, changing its direction or speed.
--------
The forces excerted by each are equal because they cancel each o

In [ ]:
for a in root.iter('Annotation'):
  l = a.attrib['Label'].split('|')
  print(l)
  ls = [i[-3:] == '(1)' for i in l]
  print(ls.index(True))
  print('---------------------------------------')

['correct(0)', 'correct_but_incomplete(1)', 'contradictory(0)', 'incorrect(0)']
1
---------------------------------------
['correct(1)', 'correct_but_incomplete(0)', 'contradictory(0)', 'incorrect(0)']
0
---------------------------------------
['correct(0)', 'correct_but_incomplete(0)', 'contradictory(0)', 'incorrect(1)']
3
---------------------------------------
['correct(0)', 'correct_but_incomplete(0)', 'contradictory(0)', 'incorrect(1)']
3
---------------------------------------
['correct(0)', 'correct_but_incomplete(0)', 'contradictory(0)', 'incorrect(1)']
3
---------------------------------------
['correct(1)', 'correct_but_incomplete(0)', 'contradictory(0)', 'incorrect(0)']
0
---------------------------------------
['correct(1)', 'correct_but_incomplete(0)', 'contradictory(0)', 'incorrect(0)']
0
---------------------------------------
['correct(0)', 'correct_but_incomplete(0)', 'contradictory(0)', 'incorrect(1)']
3
---------------------------------------
['correct(1)', 'correct_

In [ ]:
for a in root.iter('ReferenceAnswers'):
  l = a.text.split('\n')
  l = [i for i in l if i != '' ]
  print(l)
  l = [s[s.find(next(filter(str.isalpha, s))):] for s in l]
  print(l)
  print('---------------------------------------')

['1:  Since the windshield exerts a force on the mosquito, which we can call action, the mosquito exerts an equal and opposite force on the windshield, called the reaction.', '2:  The action  is the windshield squashing the mosquito, and the equal and opposite reaction is the mosquito hitting the windshield.', '3:  The force exerted by the windshield on the mosquito and the force exerted by the mosquito on the windshield are a third-law pair of action and reaction.', '4:  The force exerted by the windshield on the mosquito and the force exerted by the mosquito on the windshield are an action-reaction pair']
['Since the windshield exerts a force on the mosquito, which we can call action, the mosquito exerts an equal and opposite force on the windshield, called the reaction.', 'The action  is the windshield squashing the mosquito, and the equal and opposite reaction is the mosquito hitting the windshield.', 'The force exerted by the windshield on the mosquito and the force exerted by the

Import the libaraies, customzing our dataset and dataloaders

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler , BatchSampler
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

In [ ]:
class SeqData (Dataset):
  def __init__(self):
    self.labels = self.get_labels()
    self.Ans = self.get_ans()
    self.Refs = self.get_refs()
    self.Qs = self.get_qs()

  def get_labels(self):
    # 0 1 2 3
    #['correct', 'correct_but_incomplete', 'contradictory', 'incorrect']
    labels = list()
    for a in root.iter('Annotation'):
      l = a.attrib['Label'].split('|')
      ls = [i[-3:] == '(1)' for i in l]
      ls = ls.index(True)
      labels.append(ls)
    return labels
  
  def get_qs (self):
    Qs = list()
    for q in root.iter('Question'):
      Qs.append(q.text)
    return Qs
  
  def get_refs (self):
    Refs = list()
    for a in root.iter('ReferenceAnswers'):
      l = a.text.split('\n')
      l = [i for i in l if i != '' ]
      l = [s[s.find(next(filter(str.isalpha, s))):] for s in l]
      Refs.append(l)
    
    return Refs
  
  def get_ans(self):
    Ans = list()
    for a in root.iter('Answer'):
      Ans.append(a.text)
    return Ans
  
  def __len__(self):
    return len(self.Qs)
  
  def __getitem__(self, index):
    label = self.labels[index]
    return [self.Qs[index] ,  self.Ans[index] , self.Refs[index]] , label


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler , BatchSampler
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

In [ ]:
def split_rand(dataset,batch_size, split_ratio=0.8, seed=42, shuffle=True):
    import math
    num_entries = len(dataset)
    indices = list(range(num_entries))
    np.random.seed(seed)
    np.random.shuffle(indices)
    split = int(math.floor(split_ratio * num_entries))
    train_idx, valid_idx = indices[:split], indices[split:]
    
    
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    def collate_fn(batch):
      return tuple(zip(*batch))

    train_loader = DataLoader(
            dataset, sampler=train_sampler,
            batch_size=batch_size, collate_fn = collate_fn)
    
    valid_loader = DataLoader(
            dataset, sampler=valid_sampler,
            batch_size=batch_size, collate_fn = collate_fn)

    return train_loader, valid_loader

In [ ]:
dataset = SeqData()
train_loader , test_loader = split_rand(dataset, batch_size = 8)

In [ ]:
dataset[5][0]

['How will the meteor move after the rocket stops pushing?',
 'The meteor will continue in a straight line with a constant velocity until another force acts upon it. This is due to newtons first law of motion, objects at constant velocity will stay so unless a net force acts upon it, changing its direction or speed.',
 ['When the rocket stops pushing, no forces are acting on the meteor anymore and therefore will move with constant velocity in a straight line.',
  'When the rocket stops pushing, the meteor will move with constant velocity in a straight line.',
  'The meteor will move with constant velocity in a straight line.',
  'The meteor will move at a constant speed in the same direction in a straight line.']]

In [ ]:
next(iter(test_loader))[0][0] # q , a ,refs

['What forces are acting on the box?',
 'A frictional force going in the opposite direction as the push force that the mover is putting on the box.',
 ['The forces are the force of the mover pushing on the box, the force of friction from the floor on the box, the gravitational force from the earth on the box, and the normal force from the floor on the box.',
  'The forces are mover on box, friction from floor on box, the gravity from earth on box, and the normal from floor on box.',
  'push, friction, gravity, normal']]

Install sentance transformers libary for sentance embedding in deep learning model

In [ ]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 4.5 MB/s 
     |████████████████████████████████| 4.4 MB 14.7 MB/s 
     |████████████████████████████████| 1.2 MB 33.3 MB/s 
     |████████████████████████████████| 86 kB 3.8 MB/s 
     |████████████████████████████████| 596 kB 46.7 MB/s 
     |████████████████████████████████| 6.6 MB 35.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=9e8fb5343a0e2da1a66d46f341812d1b160de4e48f948eb5be04bdcc1a8611c5
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import SentenceTransformer

Buliding deep learning model:-

first MLP for classifiy
and Model 

In [ ]:
class MLP(nn.Module):
    """MLP with linear output"""
    def __init__(self, num_layers, input_dim, hidden_dim, output_dim):
        """MLP layers construction
        Paramters
        ---------
        num_layers: int
            The number of linear layers
        input_dim: int
            The dimensionality of input features
        hidden_dim: int
            The dimensionality of hidden units at ALL layers
        output_dim: int
            The number of classes for prediction
        """
        super(MLP, self).__init__()
        self.linear_or_not = True  # default is linear model
        self.num_layers = num_layers
        self.output_dim = output_dim

        if num_layers < 1:
            raise ValueError("number of layers should be positive!")
        elif num_layers == 1:
            # Linear model
            self.linear = nn.Linear(input_dim, output_dim)
        else:
            # Multi-layer model
            self.linear_or_not = False
            self.linears = torch.nn.ModuleList()
            #self.batch_norms = torch.nn.ModuleList()

            self.linears.append(nn.Linear(input_dim, hidden_dim))
            for layer in range(num_layers - 2):
                self.linears.append(nn.Linear(hidden_dim, hidden_dim))
            self.linears.append(nn.Linear(hidden_dim, output_dim))

            #for layer in range(num_layers - 1):
             #   self.batch_norms.append(nn.BatchNorm1d((hidden_dim)))

    def forward(self, x):
        if self.linear_or_not:
            # If linear model
            return self.linear(x)
        else:
            # If MLP
            h = x
            for i in range(self.num_layers - 1):
                h = F.relu(self.linears[i](h))
            return self.linears[-1](h)

In [ ]:
class Model(nn.Module):
  def __init__(self , num_layers = 3, input_dim = 384*3 , hidden_dim = 1024, output_dim = 4):
      super(Model,self).__init__()
      self.embedding  = SentenceTransformer('all-MiniLM-L6-v2')
      self.classifer = MLP(num_layers, input_dim, hidden_dim, output_dim)

  def forward (self , D):
    input = None
    for data in D:
      q = data[0]
      ans = data[1]
      refs = data[2]

      emb_q = torch.unsqueeze(torch.tensor(self.embedding.encode(q), dtype = torch.float), dim = 0) #(1,384)
      emb_ans = torch.unsqueeze(torch.tensor(self.embedding.encode(ans), dtype = torch.float), dim = 0)#(1,384)
      emb_refs = torch.tensor(self.embedding.encode(refs), dtype = torch.float)#(n,384)
      pool_emb_refs = torch.unsqueeze(torch.mean(emb_refs, dim = 0) , dim = 0) #(1,384)
      embs = torch.cat((emb_q , emb_ans , pool_emb_refs) , dim = 1) #(1,3*384)
      if input == None:
        input = embs
      else:
        input = torch.cat((input, embs) , dim = 0)

    return self.classifer(input)


Now, training and testing out model

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def train(net, trainloader, optimizer, criterion):
    net.train()

    running_loss = 0
    total_iters = len(trainloader)

    for idx , data  in enumerate(trainloader):

        x, labels = data
        outputs = net(x)
        labels = torch.tensor(labels , dtype = torch.long)

        loss = criterion(outputs, labels)
        running_loss += loss.item()

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    running_loss = running_loss / total_iters

    return running_loss

def eval_net(net, dataloader, criterion):
    net.eval()

    total = 0
    total_loss = 0
    total_correct = 0
    avg_f1 = 0
    all_predicted , all_labels = [], []
    for idx ,  data in enumerate(dataloader):
        x, labels = data
        labels = torch.tensor(labels , dtype = torch.long)
        total += len(labels)
        outputs = net(x)
        _, predicted = torch.max(outputs.data, 1)

        total_correct += (predicted == labels.data).sum().item()

        avg_f1 += f1_score(labels.data.detach().numpy(), predicted.detach().numpy(), average='weighted')
          
        loss = criterion(outputs, labels)
        total_loss += loss.item() * len(labels)

    loss, acc , f1 = 1.0*total_loss / total, 1.0*total_correct / total , avg_f1 *1.0 / 8


    return loss, acc, f1

In [ ]:
model = Model()

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for _ in range(3):
    print('running_loss ',train(model, train_loader, optimizer, criterion))
    print('------------------------------------------------------------------------')

running_loss  1.2540064560042488
------------------------------------------------------------------------
running_loss  1.1205350630813176
------------------------------------------------------------------------
running_loss  1.0439523100852965
------------------------------------------------------------------------


In [ ]:
loss, acc, f1 = eval_net(model, test_loader, criterion)

In [ ]:
'Accuracy is ',acc * 100.0

('Accuracy is ', 59.44444444444444)

In [ ]:
'f1 score is ',f1

('f1 score is ', 1.71875)

We can conclude that our deep model accuracy > Evaluation Dataset (DT-Grade) and Word Weighting Approach towards
Constructed Short Answers Assessment in Tutorial Dialogue Context model accuracy. (51.6 > 49.33)

So, our model is better than the paper model in accuracy.

Our f1 score is not good, but we can improve the model such as using stronger embedding model, or using imblance data methods to enhance the dataset and so on...